In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00


In [2]:
#need to install TensorFlow 2.0 or PyTorch beforehand

!pip install tensorflow

In [3]:
import pandas as pd
#data = pd.read_json('test.json')
data = pd.read_json('train.json')
data.head()

,reviews,sentiments
0,I bought this belt for my daughter in-law for ...,1
1,The size was perfect and so was the color. It...,1
2,"Fits and feels good, esp. for doing a swim rac...",1
3,These socks are absolutely the best. I take pi...,1
4,Thank you so much for the speedy delivery they...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7401 entries, 0 to 7400
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviews     7401 non-null   object
 1   sentiments  7401 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 115.8+ KB


In [5]:
data['reviews'] = data['reviews'].astype('string')

In [7]:
from transformers import DistilBertTokenizer, pipeline
from sklearn.model_selection import train_test_split

# Load your data
# data = pd.read_json('train.json')
# data['reviews'] = data['reviews'].astype('string')
#data = data[:500]

# First split: Separate out the training data
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42, stratify=data['sentiments'])

# Second split: Divide the remaining data into validation and test sets
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data['sentiments'])

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize and truncate the reviews
MAX_LENGTH = 512
tokenized_data = tokenizer(test_data['reviews'].tolist(), truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors="tf")

# Convert tokenized data back to text (this will have sequences truncated to MAX_LENGTH)
truncated_reviews = tokenizer.batch_decode(tokenized_data["input_ids"].numpy(), skip_special_tokens=True)

# Use the sentiment pipeline on the truncated data
sentiment_pipeline = pipeline("sentiment-analysis", framework="tf")
results = sentiment_pipeline(truncated_reviews)

print(results)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


[{'label': 'POSITIVE', 'score': 0.5950886607170105}, {'label': 'NEGATIVE', 'score': 0.9973800778388977}, {'label': 'POSITIVE', 'score': 0.9994890689849854}, {'label': 'POSITIVE', 'score': 0.9991355538368225}, {'label': 'NEGATIVE', 'score': 0.9964409470558167}, {'label': 'POSITIVE', 'score': 0.9998249411582947}, {'label': 'POSITIVE', 'score': 0.9994188547134399}, {'label': 'POSITIVE', 'score': 0.9998853206634521}, {'label': 'POSITIVE', 'score': 0.9998414516448975}, {'label': 'POSITIVE', 'score': 0.999749481678009}, {'label': 'POSITIVE', 'score': 0.9665238261222839}, {'label': 'NEGATIVE', 'score': 0.9466057419776917}, {'label': 'POSITIVE', 'score': 0.9981932044029236}, {'label': 'NEGATIVE', 'score': 0.9930649399757385}, {'label': 'POSITIVE', 'score': 0.999416708946228}, {'label': 'NEGATIVE', 'score': 0.9923690557479858}, {'label': 'POSITIVE', 'score': 0.9998096823692322}, {'label': 'NEGATIVE', 'score': 0.9989010095596313}, {'label': 'POSITIVE', 'score': 0.9994693398475647}, {'label': 'NE

In [8]:
# # Extracting sentiment labels
# predicted_labels = [result['label'] for result in results]

# # Convert to required format
# submission_df = pd.DataFrame({'reviews': data['reviews'], 'predicted_sentiments': predicted_labels})

# # Save to CSV
# submission_df.to_csv('submission.csv', index=False)


ValueError: ignored

In [9]:
predicted_labels = [1 if r['label'] == 'POSITIVE' else 0 for r in results]

true_labels = test_data['sentiments'].tolist()

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)

from sklearn.metrics import roc_curve, auc

# Assuming you have probability scores in your results
predicted_probabilities = [r['score'] if r['label'] == 'POSITIVE' else 1 - r['score'] for r in results]
fpr, tpr, thresholds = roc_curve(true_labels, predicted_probabilities)
roc_auc = auc(fpr, tpr)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Printing the results
print("Accuracy:", accuracy)
print("ROC AUC:", roc_auc)
print("Confusion Matrix:\n", cm)

Accuracy: 0.8433843384338434
ROC AUC: 0.9279982397556366
Confusion Matrix:
 [[142  21]
 [153 795]]


In [10]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

import pandas as pd
from sklearn.model_selection import train_test_split

# Now, extract texts and labels for training, validation, and test sets
train_texts = train_data['reviews'].tolist()
train_labels = train_data['sentiments'].tolist()

val_texts = val_data['reviews'].tolist()
val_labels = val_data['sentiments'].tolist()

# test_texts = test_data['reviews'].tolist()
# test_labels = test_data['sentiments'].tolist()

"""
import pandas as pd
#data = pd.read_json('test.json')
data = pd.read_json('train.json')
data['reviews'] = data['reviews'].astype('string')
data = data[:100]

import pandas as pd
#data = pd.read_json('test.json')
data1 = pd.read_json('train.json')
data1['reviews'] = data1['reviews'].astype('string')
data1 = data1[-50:]
"""

# Load the pre-trained DistilBERT tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

#train_texts = data['reviews'].tolist();
#val_texts = data1['reviews'].tolist();

# Tokenize and encode your data here
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

#train_labels = data['sentiments'].tolist()
#val_labels = data1['sentiments'].tolist()

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels))

# Prepare the datasets for training
train_dataset = train_dataset.shuffle(len(train_texts)).batch(16)
val_dataset = val_dataset.batch(64)

# Define loss and optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=3, validation_data=val_dataset)



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
324/324 [==============================] - 331s 939ms/step - loss: 0.2227 - accuracy: 0.9143 - val_loss: 0.1724 - val_accuracy: 0.9369
Epoch 2/3
324/324 [==============================] - 301s 930ms/step - loss: 0.0937 - accuracy: 0.9666 - val_loss: 0.1816 - val_accuracy: 0.9378
Epoch 3/3
324/324 [==============================] - 323s 997ms/step - loss: 0.0398 - accuracy: 0.9867 - val_loss: 0.1704 - val_accuracy: 0.9459


In [41]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf
import numpy as np

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Load your trained model (replace 'my_saved_model' with the actual path)
#model = TFDistilBertForSequenceClassification.from_pretrained('my_saved_model')

# Assume test_data['reviews'] contains the new reviews you want to analyze
# Tokenize the reviews
tokenized_data = tokenizer(test_data['reviews'].tolist(), truncation=True, padding='max_length', max_length=512, return_tensors="tf")

# Prepare the tokenized data for the model
input_ids = tokenized_data['input_ids']
attention_mask = tokenized_data['attention_mask']

# Define a batch size
batch_size = 32  # You can adjust this based on your available memory

# Predict in batches
predictions = []
for i in range(0, len(input_ids), batch_size):
    batch_input_ids = input_ids[i:i + batch_size]
    batch_attention_mask = attention_mask[i:i + batch_size]
    batch_predictions = model.predict({'input_ids': batch_input_ids, 'attention_mask': batch_attention_mask})

    # Ensure that batch_predictions contains actual numerical predictions
    batch_predictions = batch_predictions['logits']  # Assuming the output is in 'logits' field

    predictions.extend(batch_predictions)

# Convert predictions list to a numpy array
predictions = np.array(predictions, dtype=np.float32)  # Ensure predictions are in a valid data type

# Check the shape of predictions
print("Shape of predictions:", predictions.shape)

# # If predictions are logits, convert them to probabilities using softmax
# # Adjust the axis based on the shape of your predictions
probabilities = tf.nn.softmax(predictions, axis=-1)

# # Convert probabilities to label predictions
predicted_labels = tf.argmax(probabilities, axis=-1).numpy()

# # Print the predicted labels
print(predicted_labels)

1/1 [==============================] - 0s 490ms/step
Shape of predictions: (1111, 2)
[1 1 1 ... 1 1 1]


In [42]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

true_labels = test_data['sentiments'].tolist()
# Convert probabilities to positive class probabilities
positive_class_probabilities = probabilities[:, 1]

# Accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)

# ROC AUC
# Note: ROC AUC requires the probability scores of the positive class, not the label predictions.
roc_auc = roc_auc_score(true_labels, positive_class_probabilities)
print("ROC AUC:", roc_auc)

# Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", cm)



Accuracy: 0.9414941494149415
ROC AUC: 0.961902358209728
Confusion Matrix:
 [[123  40]
 [ 25 923]]


In [43]:
data = pd.read_json('test.json')
data['reviews'] = data['reviews'].astype('string')

tokenized_data = tokenizer(data['reviews'].tolist(), truncation=True, padding='max_length', max_length=512, return_tensors="tf")

# Prepare the tokenized data for the model
input_ids = tokenized_data['input_ids']
attention_mask = tokenized_data['attention_mask']

# Define a batch size
batch_size = 32  # You can adjust this based on your available memory

# Predict in batches
predictions = []
for i in range(0, len(input_ids), batch_size):
    batch_input_ids = input_ids[i:i + batch_size]
    batch_attention_mask = attention_mask[i:i + batch_size]
    batch_predictions = model.predict({'input_ids': batch_input_ids, 'attention_mask': batch_attention_mask})

    # Ensure that batch_predictions contains actual numerical predictions
    batch_predictions = batch_predictions['logits']  # Assuming the output is in 'logits' field

    predictions.extend(batch_predictions)

# Convert predictions list to a numpy array
predictions = np.array(predictions, dtype=np.float32)  # Ensure predictions are in a valid data type

# Check the shape of predictions
print("Shape of predictions:", predictions.shape)

# # If predictions are logits, convert them to probabilities using softmax
# # Adjust the axis based on the shape of your predictions
probabilities = tf.nn.softmax(predictions, axis=-1)

# # Convert probabilities to label predictions
predicted_labels = tf.argmax(probabilities, axis=-1).numpy()

# # Print the predicted labels
print(predicted_labels)

1/1 [==============================] - 1s 502ms/step
Shape of predictions: (1851, 2)
[0 1 1 ... 1 1 1]


In [44]:
import pandas as pd

# Assuming test_data and predicted_labels are already defined
# test_data should be the DataFrame that was used for the predictions
# predicted_labels should be the array of predicted sentiment labels

# Add the predicted labels to the test_data DataFrame
data['predicted_sentiments'] = predicted_labels

# Select only the 'reviews' and 'predicted_sentiments' for the CSV
submission_df = data[['reviews', 'predicted_sentiments']]

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

In [ ]:
# true_labels = data['sentiments'].tolist()

# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(true_labels, predicted_labels)

# from sklearn.metrics import roc_curve, auc

# # Assuming you have probability scores in your results
# #predicted_probabilities = [r['score'] if r['label'] == 'POSITIVE' else 1 - r['score'] for r in results]
# #fpr, tpr, thresholds = roc_curve(true_labels, probabilities)
# #roc_auc = auc(fpr, tpr)

# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(true_labels, predicted_labels)

# # Printing the results
# print("Accuracy:", accuracy)
# #print("ROC AUC:", roc_auc)
# print("Confusion Matrix:\n", cm)

In [ ]:
"""
from transformers import AutoTokenizer, pipeline

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")

# Tokenize and truncate the reviews
MAX_LENGTH = 512
tokenized_data = tokenizer(data['reviews'].tolist(), truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors="tf")

# Convert tokenized data back to text (this will have sequences truncated to MAX_LENGTH)
truncated_reviews = tokenizer.batch_decode(tokenized_data["input_ids"].numpy(), skip_special_tokens=True)

# Use the sentiment pipeline on the truncated data
sentiment_pipeline = pipeline(model="siebert/sentiment-roberta-large-english", framework="tf")
results = sentiment_pipeline(truncated_reviews)

print(results)
"""

In [ ]:
# num_entries = len(results)
# print(f"There are {num_entries} entries in the results.")